In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from sklearn.metrics import (
    make_scorer,
    mean_absolute_error,
    mean_squared_error,
    r2_score
)
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import  MLPRegressor
from sklearn.svm import SVR
import numpy as np

In [3]:
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

METRICS_REGRESSION = {
    "MAE": make_scorer(mean_absolute_error),
    "MSE": make_scorer(mean_squared_error),
    "RMSE": make_scorer(root_mean_squared_error),
    "R2": make_scorer(r2_score)
}

In [4]:
df = pd.read_csv("../cenario3_engine.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2217 entries, 0 to 2216
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   model_year  2217 non-null   int64  
 1   KM          2217 non-null   int64  
 2   HP          2217 non-null   float64
 3   Litros      2217 non-null   float64
 4   Cilindros   2217 non-null   float64
 5   price_eur   2217 non-null   int64  
dtypes: float64(3), int64(3)
memory usage: 104.1 KB


In [6]:
categorical_columns = df.select_dtypes(include='object').columns

for column in categorical_columns:
    df[column] = df[column].astype('category').cat.codes

In [7]:
X, y = df.drop("price_eur", axis=1), df["price_eur"]

In [8]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

Decision Tree

In [9]:
dt = DecisionTreeRegressor(max_depth=3, random_state=1234)
scores_dt = cross_validate(dt, X, y, cv=splitter, scoring=METRICS_REGRESSION)
dt_scores = pd.DataFrame(scores_dt)
pd.DataFrame(dt_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_R2
0,0.003596,0.002061,8128.692253,1.168338e+08,10794.047986,0.638635


Random Forest

In [10]:
rf = RandomForestRegressor(random_state=1234)
scores_rf = cross_validate(rf, X, y, cv=splitter, scoring=METRICS_REGRESSION)
rf_scores = pd.DataFrame(scores_rf)
pd.DataFrame(rf_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_R2
0,1.153489,0.009134,5553.624841,6.274687e+07,7911.049464,0.806108


Redes Neuronais

In [13]:
nn = MLPRegressor(hidden_layer_sizes=(50,50), max_iter=200, random_state=1234)
scores_nn = cross_validate(nn, X, y, cv=splitter, scoring=METRICS_REGRESSION)
nn_scores = pd.DataFrame(scores_nn)
pd.DataFrame(nn_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_R2
0,1.506275,0.002012,7683.262637,1.079274e+08,10381.460708,0.666549


Support Vector Machine

In [12]:
svm = SVR()
scores_svm = cross_validate(svm, X, y, cv=splitter, scoring=METRICS_REGRESSION)
svm_scores = pd.DataFrame(scores_svm)
pd.DataFrame(svm_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_R2
0,0.130647,0.032865,14089.933952,3.317930e+08,18208.407993,-0.023956
